<a href="https://colab.research.google.com/github/fd4dg4br1/Pesquisa-SUS/blob/main/painel_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')
OUTPUT_DIR = os.path.join(BASE_DIR, 'data/final')

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- FUNÇÕES DE CATEGORIZAÇÃO ---

def classificar_causa_obito(cid):
    """Classifica o CID-10 do SIM em grupos analíticos"""
    cid = str(cid).upper().strip()
    if cid.startswith(('X8', 'X9', 'Y0')): return 'Homicidio'
    if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Suicidio'
    if cid.startswith(('V')): return 'Transito'
    return 'Outras Causas Externas'

def classificar_crime_sinesp(crime):
    """Agrupa crimes do SINESP (nomes podem variar)"""
    crime = str(crime).lower()
    if 'homicidio' in crime or 'homicídio' in crime: return 'Homicidio Doloso'
    if 'latrocinio' in crime or 'latrocínio' in crime: return 'Latrocinio'
    if 'lesao' in crime and 'morte' in crime: return 'Lesao Corp Seg Morte'
    if 'estupro' in crime: return 'Estupro'
    if 'roubo' in crime: return 'Roubo (Geral)'
    if 'furto' in crime: return 'Furto (Geral)'
    return 'Outros Crimes'

# --- ORQUESTRADOR ---

def gerar_painel_analitico():
    print("🚀 Gerando Painel Analítico Detalhado (2014-2025)...")

    dfs_to_merge = []

    # 1. POPULAÇÃO
    try:
        print("📊 Processando População...")
        f_pop = [f for f in os.listdir(PROCESSED_DIR) if 'pop' in f.lower() and 'csv' in f][0]
        df_pop = pd.read_csv(os.path.join(PROCESSED_DIR, f_pop))
        df_pop_anual = df_pop.groupby('ano')['populacao'].sum().reset_index()
        dfs_to_merge.append(df_pop_anual)
    except Exception as e:
        print(f"❌ Erro População: {e}")
        return

    # 2. SIM (Mortalidade Detalhada)
    try:
        print("💀 Processando SIM (Categorizando CIDs)...")
        f_sim = [f for f in os.listdir(PROCESSED_DIR) if 'sim_' in f.lower() and 'csv' in f][0]
        df_sim = pd.read_csv(os.path.join(PROCESSED_DIR, f_sim))

        df_sim['ano'] = pd.to_datetime(df_sim['data_obito']).dt.year
        df_sim['categoria'] = df_sim['causa_basica_cid10'].apply(classificar_causa_obito)

        # Pivota: Linhas viram Colunas (Ex: total_Homicidio, total_Suicidio)
        df_sim_pivot = df_sim.pivot_table(index='ano', columns='categoria', values='data_obito', aggfunc='count').fillna(0)
        df_sim_pivot.columns = [f"obitos_{c.lower().replace(' ', '_')}" for c in df_sim_pivot.columns]
        df_sim_pivot = df_sim_pivot.reset_index()

        dfs_to_merge.append(df_sim_pivot)
    except Exception as e:
        print(f"⚠️ Erro SIM: {e}")

    # 3. SINESP (Segurança Detalhada)
    try:
        print("🚓 Processando SINESP (Categorizando Crimes)...")
        f_sinesp = [f for f in os.listdir(PROCESSED_DIR) if 'sinesp_' in f.lower() and 'csv' in f][0]
        df_sinesp = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinesp))

        # Aplica classificação simplificada para garantir colunas consistentes
        df_sinesp['grupo_crime'] = df_sinesp['tipo_crime'].apply(classificar_crime_sinesp)

        df_sinesp_pivot = df_sinesp.pivot_table(index='ano', columns='grupo_crime', values='quantidade', aggfunc='sum').fillna(0)
        df_sinesp_pivot.columns = [f"ocorrencias_{c.lower().replace(' ', '_').replace('(', '').replace(')', '')}" for c in df_sinesp_pivot.columns]
        df_sinesp_pivot = df_sinesp_pivot.reset_index()

        # Soma total de Crimes Violentos Letais Intencionais (CVLI) se possível
        cols_cvli = [c for c in df_sinesp_pivot.columns if 'homicidio' in c or 'latrocinio' in c or 'lesao' in c]
        if cols_cvli:
            df_sinesp_pivot['ocorrencias_cvli_total'] = df_sinesp_pivot[cols_cvli].sum(axis=1)

        dfs_to_merge.append(df_sinesp_pivot)
    except Exception as e:
        print(f"⚠️ Erro SINESP: {e}")

    # 4. SIH (Internações e Custos)
    try:
        print("🏥 Processando SIH...")
        f_sih = [f for f in os.listdir(PROCESSED_DIR) if 'sih_' in f.lower() and 'csv' in f][0]
        df_sih = pd.read_csv(os.path.join(PROCESSED_DIR, f_sih))
        df_sih['ano'] = pd.to_datetime(df_sih['data_internacao']).dt.year

        df_sih_anual = df_sih.groupby('ano').agg(
            internacoes_totais=('valor_total', 'count'),
            custo_hospitalar_total=('valor_total', 'sum')
        ).reset_index()
        dfs_to_merge.append(df_sih_anual)
    except Exception as e:
        print(f"⚠️ Erro SIH: {e}")

    # 5. SINAN (Notificações)
    try:
        print("🤕 Processando SINAN...")
        f_sinan = [f for f in os.listdir(PROCESSED_DIR) if 'sinan_' in f.lower() and 'csv' in f][0]
        df_sinan = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinan))
        df_sinan['ano'] = pd.to_datetime(df_sinan['data_notificacao']).dt.year

        df_sinan_anual = df_sinan.groupby('ano').size().reset_index(name='notificacoes_violencia')
        dfs_to_merge.append(df_sinan_anual)
    except Exception as e:
        print(f"⚠️ Erro SINAN: {e}")

    # --- INTEGRAÇÃO ---
    print("\n🔗 Unificando tudo...")
    df_mestre = dfs_to_merge[0]
    for df in dfs_to_merge[1:]:
        df_mestre = pd.merge(df_mestre, df, on='ano', how='left')

    df_mestre = df_mestre.fillna(0)

    # --- CÁLCULO DE TAXAS (INDICADORES) ---
    print("🧮 Calculando Taxas por 100 mil hab...")

    # Taxas de Homicídio (Comparando SIM vs SINESP)
    if 'obitos_homicidio' in df_mestre.columns:
        df_mestre['taxa_homicidio_saude'] = (df_mestre['obitos_homicidio'] / df_mestre['populacao']) * 100000

    if 'ocorrencias_homicidio_doloso' in df_mestre.columns:
        df_mestre['taxa_homicidio_policia'] = (df_mestre['ocorrencias_homicidio_doloso'] / df_mestre['populacao']) * 100000

    # Taxa de Suicídio
    if 'obitos_suicidio' in df_mestre.columns:
        df_mestre['taxa_suicidio'] = (df_mestre['obitos_suicidio'] / df_mestre['populacao']) * 100000

    # Taxa de Notificação de Violência (SINAN)
    if 'notificacoes_violencia' in df_mestre.columns:
        df_mestre['taxa_notificacao_violencia'] = (df_mestre['notificacoes_violencia'] / df_mestre['populacao']) * 100000

    # Arredondamento
    df_mestre = df_mestre.round(2)

    # Ordenação de colunas para facilitar leitura
    cols_order = ['ano', 'populacao'] + [c for c in df_mestre.columns if 'taxa' in c] + [c for c in df_mestre.columns if c not in ['ano', 'populacao'] and 'taxa' not in c]
    df_mestre = df_mestre[cols_order]

    # Salva
    path_final = os.path.join(OUTPUT_DIR, 'PAINEL_ANALITICO_FINAL.csv')
    df_mestre.to_csv(path_final, index=False)

    print(f"\n🏆 SUCESSO! Painel Analítico salvo em: {path_final}")
    print("\n🔎 Amostra das Colunas Geradas:")
    print(df_mestre.columns.tolist())
    print("\n🔎 Primeiras linhas:")
    print(df_mestre.head())

    return df_mestre

if __name__ == "__main__":
    df_final = gerar_painel_analitico()

🚀 Gerando Painel Analítico Detalhado (2014-2025)...
📊 Processando População...
💀 Processando SIM (Categorizando CIDs)...
🚓 Processando SINESP (Categorizando Crimes)...
🏥 Processando SIH...


/tmp/ipython-input-3726801874.py:95: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sih = pd.read_csv(os.path.join(PROCESSED_DIR, f_sih))


🤕 Processando SINAN...

🔗 Unificando tudo...
🧮 Calculando Taxas por 100 mil hab...

🏆 SUCESSO! Painel Analítico salvo em: /content/drive/MyDrive/criminalidade_df/data/final/PAINEL_ANALITICO_FINAL.csv

🔎 Amostra das Colunas Geradas:
['ano', 'populacao', 'taxa_homicidio_saude', 'taxa_homicidio_policia', 'taxa_suicidio', 'taxa_notificacao_violencia', 'obitos_homicidio', 'obitos_outras_causas_externas', 'obitos_suicidio', 'obitos_transito', 'ocorrencias_estupro', 'ocorrencias_furto_geral', 'ocorrencias_homicidio_doloso', 'ocorrencias_latrocinio', 'ocorrencias_outros_crimes', 'ocorrencias_roubo_geral', 'ocorrencias_cvli_total', 'internacoes_totais', 'custo_hospitalar_total', 'notificacoes_violencia']

🔎 Primeiras linhas:
    ano  populacao  taxa_homicidio_saude  taxa_homicidio_policia  \
0  2014    2852372                 30.29                    0.00   
1  2015    2914830                 25.94                   44.26   
2  2016    2977216                 26.33                   49.95   
3 

In [4]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')
OUTPUT_DIR = os.path.join(BASE_DIR, 'data/final')

os.makedirs(OUTPUT_DIR, exist_ok=True)


def classificar_causa_obito(cid):
    """Classifica o CID-10 do SIM (Agora inclui F - Mental)"""
    cid = str(cid).upper().strip()
    if cid.startswith('F'): return 'Transtornos Mentais'
    if cid.startswith(('X8', 'X9', 'Y0')): return 'Homicidio'
    if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Suicidio'
    if cid.startswith(('V')): return 'Transito'
    return 'Outras Causas Externas'

def classificar_diagnostico_sih(cid):
    """Classifica o CID-10 do SIH (Separa Mental de Trauma e Agressão)"""
    cid = str(cid).upper().strip()
    if cid.startswith('F'): return 'Mental' # Ex: Esquizofrenia, Álcool, Drogas
    if cid.startswith(('X8', 'X9', 'Y0')): return 'Agressao' # Vítima de violência explícita
    if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Autoprovocada'
    if cid.startswith(('S', 'T')): return 'Traumas_Lesoes' # Fraturas, ferimentos (consequências)
    return 'Acidentes_Outros'

def classificar_crime_sinesp(crime):
    """Agrupa crimes do SINESP"""
    crime = str(crime).lower()
    if 'homicidio' in crime or 'homicídio' in crime: return 'Homicidio Doloso'
    if 'latrocinio' in crime or 'latrocínio' in crime: return 'Latrocinio'
    if 'lesao' in crime and 'morte' in crime: return 'Lesao Corp Seg Morte'
    if 'estupro' in crime: return 'Estupro'
    if 'roubo' in crime: return 'Roubo (Geral)'
    if 'furto' in crime: return 'Furto (Geral)'
    return 'Outros Crimes'

# --- ORQUESTRADOR ---

def gerar_painel_analitico_completo():
    print("🚀 Gerando Painel Analítico Completo (Violência + Saúde Mental)...")

    dfs_to_merge = []

    # 1. POPULAÇÃO
    try:
        print("📊 Processando População...")
        f_pop = [f for f in os.listdir(PROCESSED_DIR) if 'pop' in f.lower() and 'csv' in f][0]
        df_pop = pd.read_csv(os.path.join(PROCESSED_DIR, f_pop))
        df_pop_anual = df_pop.groupby('ano')['populacao'].sum().reset_index()
        dfs_to_merge.append(df_pop_anual)
    except Exception as e:
        print(f"❌ Erro População: {e}")
        return

    # 2. SIM (Mortalidade: Violência + Mental)
    try:
        print("💀 Processando SIM (Categorizando CIDs)...")
        # Busca o arquivo mais recente/completo
        f_sim_candidates = [f for f in os.listdir(PROCESSED_DIR) if 'sim_' in f.lower() and 'csv' in f]
        # Prioriza arquivos com 'completa' no nome, senão pega qualquer um
        f_sim = next((f for f in f_sim_candidates if 'externas' in f), f_sim_candidates[0] if f_sim_candidates else None)

        if f_sim:
            print(f"   Arquivo SIM utilizado: {f_sim}")
            df_sim = pd.read_csv(os.path.join(PROCESSED_DIR, f_sim))
            df_sim['ano'] = pd.to_datetime(df_sim['data_obito']).dt.year
            df_sim['categoria'] = df_sim['causa_basica_cid10'].apply(classificar_causa_obito)

            # Pivot (Linhas -> Colunas)
            df_sim_pivot = df_sim.pivot_table(index='ano', columns='categoria', values='data_obito', aggfunc='count').fillna(0)
            df_sim_pivot.columns = [f"obitos_{c.lower().replace(' ', '_')}" for c in df_sim_pivot.columns]
            df_sim_pivot = df_sim_pivot.reset_index()
            dfs_to_merge.append(df_sim_pivot)
        else:
            print("   ⚠️ Nenhum arquivo SIM encontrado.")
    except Exception as e:
        print(f"⚠️ Erro SIM: {e}")

    # 3. SIH (Internações: Violência + Trauma + Mental)
    try:
        print("🏥 Processando SIH (Separando Mental de Trauma)...")
        f_sih_candidates = [f for f in os.listdir(PROCESSED_DIR) if 'sih_' in f.lower() and 'csv' in f]
        f_sih = next((f for f in f_sih_candidates if 'completa' in f), f_sih_candidates[0] if f_sih_candidates else None)

        if f_sih:
            print(f"   Arquivo SIH utilizado: {f_sih}")
            df_sih = pd.read_csv(os.path.join(PROCESSED_DIR, f_sih))
            df_sih['ano'] = pd.to_datetime(df_sih['data_internacao']).dt.year

            # Classifica Diagnóstico Principal
            df_sih['grupo'] = df_sih['diag_principal_cid10'].apply(classificar_diagnostico_sih)

            # Pivot 1: Contagem de Internações
            df_sih_count = df_sih.pivot_table(index='ano', columns='grupo', values='valor_total', aggfunc='count').fillna(0)
            df_sih_count.columns = [f"internacoes_{c.lower()}" for c in df_sih_count.columns]

            # Pivot 2: Custo Total
            df_sih_sum = df_sih.pivot_table(index='ano', columns='grupo', values='valor_total', aggfunc='sum').fillna(0)
            df_sih_sum.columns = [f"custo_{c.lower()}" for c in df_sih_sum.columns]

            # Junta contagem e custo
            df_sih_anual = pd.merge(df_sih_count.reset_index(), df_sih_sum.reset_index(), on='ano')
            dfs_to_merge.append(df_sih_anual)
        else:
            print("   ⚠️ Nenhum arquivo SIH encontrado.")
    except Exception as e:
        print(f"⚠️ Erro SIH: {e}")

    # 4. SINESP (Segurança)
    try:
        print("🚓 Processando SINESP...")
        f_sinesp = [f for f in os.listdir(PROCESSED_DIR) if 'sinesp_' in f.lower() and 'csv' in f][0]
        df_sinesp = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinesp))
        df_sinesp['grupo_crime'] = df_sinesp['tipo_crime'].apply(classificar_crime_sinesp)

        df_sinesp_pivot = df_sinesp.pivot_table(index='ano', columns='grupo_crime', values='quantidade', aggfunc='sum').fillna(0)
        df_sinesp_pivot.columns = [f"ocorrencias_{c.lower().replace(' ', '_').replace('(', '').replace(')', '')}" for c in df_sinesp_pivot.columns]
        df_sinesp_pivot = df_sinesp_pivot.reset_index()

        dfs_to_merge.append(df_sinesp_pivot)
    except Exception as e:
        print(f"⚠️ Erro SINESP: {e}")

    # 5. SINAN (Notificações)
    try:
        print("🤕 Processando SINAN...")
        f_sinan = [f for f in os.listdir(PROCESSED_DIR) if 'sinan_' in f.lower() and 'csv' in f][0]
        df_sinan = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinan))
        df_sinan['ano'] = pd.to_datetime(df_sinan['data_notificacao']).dt.year
        df_sinan_anual = df_sinan.groupby('ano').size().reset_index(name='notificacoes_violencia')
        dfs_to_merge.append(df_sinan_anual)
    except Exception as e:
        print(f"⚠️ Erro SINAN: {e}")

    # --- INTEGRAÇÃO ---
    print("\n🔗 Unificando tudo...")
    df_mestre = dfs_to_merge[0]
    for df in dfs_to_merge[1:]:
        df_mestre = pd.merge(df_mestre, df, on='ano', how='left')

    df_mestre = df_mestre.fillna(0)

    # --- CÁLCULO DE TAXAS (INDICADORES RECOMENDADOS) ---
    print("🧮 Calculando Indicadores por 100k habitantes...")

    # 1. Homicídios (Polícia vs Saúde)
    if 'obitos_homicidio' in df_mestre.columns:
        df_mestre['taxa_homicidio_saude'] = (df_mestre['obitos_homicidio'] / df_mestre['populacao']) * 100000
    if 'ocorrencias_homicidio_doloso' in df_mestre.columns:
        df_mestre['taxa_homicidio_policia'] = (df_mestre['ocorrencias_homicidio_doloso'] / df_mestre['populacao']) * 100000

    # 2. Suicídio
    if 'obitos_suicidio' in df_mestre.columns:
        df_mestre['taxa_suicidio'] = (df_mestre['obitos_suicidio'] / df_mestre['populacao']) * 100000

    # 3. Saúde Mental (Internações)
    if 'internacoes_mental' in df_mestre.columns:
        df_mestre['taxa_internacao_mental'] = (df_mestre['internacoes_mental'] / df_mestre['populacao']) * 100000

    # 4. Internações por Agressão (Violência que precisou de hospital)
    if 'internacoes_agressao' in df_mestre.columns:
        df_mestre['taxa_internacao_agressao'] = (df_mestre['internacoes_agressao'] / df_mestre['populacao']) * 100000

    df_mestre = df_mestre.round(2)

    # Ordenação Inteligente
    cols_order = ['ano', 'populacao'] + [c for c in df_mestre.columns if 'taxa' in c] + [c for c in df_mestre.columns if c not in ['ano', 'populacao'] and 'taxa' not in c]
    df_mestre = df_mestre[cols_order]

    # Salva
    path_final = os.path.join(OUTPUT_DIR, 'PAINEL_ANALITICO_COMPLETO_FINAL.csv')
    df_mestre.to_csv(path_final, index=False)

    print(f"\n🏆 SUCESSO! Painel Final salvo em: {path_final}")
    print("\n🔎 Amostra das Colunas:")
    print(df_mestre.columns.tolist())
    print("\n🔎 Dados Recentes:")
    print(df_mestre.tail())

    return df_mestre

if __name__ == "__main__":
    df_final = gerar_painel_analitico_completo()

🚀 Gerando Painel Analítico Completo (Violência + Saúde Mental)...
📊 Processando População...
💀 Processando SIM (Categorizando CIDs)...
   Arquivo SIM utilizado: sim_mortalidade_externa_df.csv
🏥 Processando SIH (Separando Mental de Trauma)...
   Arquivo SIH utilizado: sih_internacoes_externas_df.csv


/tmp/ipython-input-4117948425.py:92: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sih = pd.read_csv(os.path.join(PROCESSED_DIR, f_sih))


🚓 Processando SINESP...
🤕 Processando SINAN...

🔗 Unificando tudo...
🧮 Calculando Indicadores por 100k habitantes...

🏆 SUCESSO! Painel Final salvo em: /content/drive/MyDrive/criminalidade_df/data/final/PAINEL_ANALITICO_COMPLETO_FINAL.csv

🔎 Amostra das Colunas:
['ano', 'populacao', 'taxa_homicidio_saude', 'taxa_homicidio_policia', 'taxa_suicidio', 'taxa_internacao_mental', 'taxa_internacao_agressao', 'obitos_homicidio', 'obitos_outras_causas_externas', 'obitos_suicidio', 'obitos_transito', 'obitos_transtornos_mentais', 'internacoes_acidentes_outros', 'internacoes_agressao', 'internacoes_autoprovocada', 'internacoes_mental', 'internacoes_traumas_lesoes', 'custo_acidentes_outros', 'custo_agressao', 'custo_autoprovocada', 'custo_mental', 'custo_traumas_lesoes', 'ocorrencias_estupro', 'ocorrencias_furto_geral', 'ocorrencias_homicidio_doloso', 'ocorrencias_latrocinio', 'ocorrencias_outros_crimes', 'ocorrencias_roubo_geral', 'notificacoes_violencia']

🔎 Dados Recentes:
    ano  populacao  t

In [10]:
import pandas as pd
import numpy as np
import os

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')
OUTPUT_DIR = os.path.join(BASE_DIR, 'data/final')

os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- FUNÇÕES DE CATEGORIZAÇÃO

def classificar_causa_obito(cid):
    """Classifica o CID-10 do SIM"""
    cid = str(cid).upper().strip()
    if cid.startswith('F'): return 'Transtornos Mentais'
    if cid.startswith(('X8', 'X9', 'Y0')): return 'Homicidio'
    if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Suicidio'
    if cid.startswith(('V')): return 'Transito'
    return 'Outras Causas Externas'

def classificar_diagnostico_sih(cid):
    """Classifica o CID-10 do SIH"""
    cid = str(cid).upper().strip()
    if cid.startswith('F'): return 'Mental'
    if cid.startswith(('X8', 'X9', 'Y0')): return 'Agressao'
    if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Autoprovocada'
    if cid.startswith(('S', 'T')): return 'Traumas_Lesoes'
    return 'Acidentes_Outros'

def classificar_crime_sinesp(crime):
    """Agrupa crimes do SINESP"""
    crime = str(crime).lower()
    if 'homicidio' in crime or 'homicídio' in crime: return 'Homicidio Doloso'
    if 'latrocinio' in crime or 'latrocínio' in crime: return 'Latrocinio'
    if 'lesao' in crime and 'morte' in crime: return 'Lesao Corp Seg Morte'
    if 'estupro' in crime: return 'Estupro'
    if 'roubo' in crime: return 'Roubo (Geral)'
    if 'furto' in crime: return 'Furto (Geral)'
    return 'Outros Crimes'

# --- ORQUESTRADOR ---

def gerar_painel_analitico_final():
    print("🚀 Gerando Painel Analítico Final...")

    dfs_to_merge = []

    # 1. POPULAÇÃO
    try:
        print("📊 Processando População...")
        f_pop = [f for f in os.listdir(PROCESSED_DIR) if 'pop' in f.lower() and 'csv' in f][0]
        df_pop = pd.read_csv(os.path.join(PROCESSED_DIR, f_pop))
        df_pop_anual = df_pop.groupby('ano')['populacao'].sum().reset_index()
        dfs_to_merge.append(df_pop_anual)
    except Exception as e:
        print(f"❌ Erro População: {e}")
        return

    # 2. SIM (Mortalidade)
    try:
        print("💀 Processando SIM...")
        # PROCURA O ARQUIVO EXATO QUE VOCÊ GEROU ('externa')
        arquivo_sim = 'sim_mortalidade_externa_df.csv'
        caminho_sim = os.path.join(PROCESSED_DIR, arquivo_sim)

        if os.path.exists(caminho_sim):
            print(f"   Arquivo SIM utilizado: {arquivo_sim}")
            df_sim = pd.read_csv(caminho_sim)
            df_sim['ano'] = pd.to_datetime(df_sim['data_obito']).dt.year

            # Aplica a classificação (Aqui ele vai separar o 'F' que está escondido no arquivo)
            df_sim['categoria'] = df_sim['causa_basica_cid10'].apply(classificar_causa_obito)

            # Pivot
            df_sim_pivot = df_sim.pivot_table(index='ano', columns='categoria', values='data_obito', aggfunc='count').fillna(0)
            df_sim_pivot.columns = [f"obitos_{c.lower().replace(' ', '_')}" for c in df_sim_pivot.columns]
            df_sim_pivot = df_sim_pivot.reset_index()
            dfs_to_merge.append(df_sim_pivot)
        else:
            print(f"   ⚠️ Arquivo {arquivo_sim} não encontrado.")
    except Exception as e:
        print(f"⚠️ Erro SIM: {e}")

    # 3. SIH (Internações)
    try:
        print("🏥 Processando SIH...")
        # PROCURA O ARQUIVO EXATO QUE VOCÊ GEROU ('externas')
        arquivo_sih = 'sih_internacoes_externas_df.csv'
        caminho_sih = os.path.join(PROCESSED_DIR, arquivo_sih)

        if os.path.exists(caminho_sih):
            print(f"   Arquivo SIH utilizado: {arquivo_sih}")
            df_sih = pd.read_csv(caminho_sih)
            df_sih['ano'] = pd.to_datetime(df_sih['data_internacao']).dt.year

            # Aplica a classificação (Vai pegar S, T e F agora)
            df_sih['grupo'] = df_sih['diag_principal_cid10'].apply(classificar_diagnostico_sih)

            # Pivot 1: Contagem
            df_sih_count = df_sih.pivot_table(index='ano', columns='grupo', values='valor_total', aggfunc='count').fillna(0)
            df_sih_count.columns = [f"internacoes_{c.lower()}" for c in df_sih_count.columns]

            # Pivot 2: Custo
            df_sih_sum = df_sih.pivot_table(index='ano', columns='grupo', values='valor_total', aggfunc='sum').fillna(0)
            df_sih_sum.columns = [f"custo_{c.lower()}" for c in df_sih_sum.columns]

            # Junta
            df_sih_anual = pd.merge(df_sih_count.reset_index(), df_sih_sum.reset_index(), on='ano')
            dfs_to_merge.append(df_sih_anual)
        else:
            print(f"   ⚠️ Arquivo {arquivo_sih} não encontrado.")
    except Exception as e:
        print(f"⚠️ Erro SIH: {e}")

    # 4. SINESP (Segurança)
    try:
        print("🚓 Processando SINESP...")
        f_sinesp = [f for f in os.listdir(PROCESSED_DIR) if 'sinesp_' in f.lower() and 'csv' in f][0]
        df_sinesp = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinesp))
        df_sinesp['grupo_crime'] = df_sinesp['tipo_crime'].apply(classificar_crime_sinesp)

        df_sinesp_pivot = df_sinesp.pivot_table(index='ano', columns='grupo_crime', values='quantidade', aggfunc='sum').fillna(0)
        df_sinesp_pivot.columns = [f"ocorrencias_{c.lower().replace(' ', '_').replace('(', '').replace(')', '')}" for c in df_sinesp_pivot.columns]
        df_sinesp_pivot = df_sinesp_pivot.reset_index()

        dfs_to_merge.append(df_sinesp_pivot)
    except Exception as e:
        print(f"⚠️ Erro SINESP: {e}")

    # 5. SINAN (Notificações)
    try:
        print("🤕 Processando SINAN...")
        f_sinan = [f for f in os.listdir(PROCESSED_DIR) if 'sinan_' in f.lower() and 'csv' in f][0]
        df_sinan = pd.read_csv(os.path.join(PROCESSED_DIR, f_sinan))
        df_sinan['ano'] = pd.to_datetime(df_sinan['data_notificacao']).dt.year
        df_sinan_anual = df_sinan.groupby('ano').size().reset_index(name='notificacoes_violencia')
        dfs_to_merge.append(df_sinan_anual)
    except Exception as e:
        print(f"⚠️ Erro SINAN: {e}")

    # --- INTEGRAÇÃO ---
    print("\n🔗 Unificando tudo...")
    if not dfs_to_merge:
        print("❌ Nada para unificar.")
        return

    df_mestre = dfs_to_merge[0]
    for df in dfs_to_merge[1:]:
        df_mestre = pd.merge(df_mestre, df, on='ano', how='left')

    df_mestre = df_mestre.fillna(0)

    # --- CÁLCULO DE TAXAS ---
    print("🧮 Calculando Indicadores por 100k habitantes...")

    # 1. Homicídios
    if 'obitos_homicidio' in df_mestre.columns:
        df_mestre['taxa_homicidio_saude'] = (df_mestre['obitos_homicidio'] / df_mestre['populacao']) * 100000
    if 'ocorrencias_homicidio_doloso' in df_mestre.columns:
        df_mestre['taxa_homicidio_policia'] = (df_mestre['ocorrencias_homicidio_doloso'] / df_mestre['populacao']) * 100000

    # 2. Suicídio
    if 'obitos_suicidio' in df_mestre.columns:
        df_mestre['taxa_suicidio'] = (df_mestre['obitos_suicidio'] / df_mestre['populacao']) * 100000

    # 3. Saúde Mental (Internações)
    if 'internacoes_mental' not in df_mestre.columns:
        df_mestre['internacoes_mental'] = 0

    df_mestre['taxa_internacao_mental'] = (df_mestre['internacoes_mental'] / df_mestre['populacao']) * 100000

    # 4. Agressão (Internações)
    if 'internacoes_agressao' not in df_mestre.columns:
        df_mestre['internacoes_agressao'] = 0

    df_mestre['taxa_internacao_agressao'] = (df_mestre['internacoes_agressao'] / df_mestre['populacao']) * 100000

    df_mestre = df_mestre.round(2)

    # Ordenação
    cols_order = ['ano', 'populacao'] + [c for c in df_mestre.columns if 'taxa' in c] + [c for c in df_mestre.columns if c not in ['ano', 'populacao'] and 'taxa' not in c]
    df_mestre = df_mestre[cols_order]

    # Salva
    path_final = os.path.join(OUTPUT_DIR, 'PAINEL_ANALITICO_FINAL.csv')
    df_mestre.to_csv(path_final, index=False)

    print(f"\n🏆 SUCESSO! Painel Final salvo em: {path_final}")
    print("\n🔎 Amostra das Colunas:")
    print(df_mestre.columns.tolist())

    # Verificação rápida se tem dados de mental/agressão
    print("\n🔎 Verificação de Conteúdo (Soma total dos anos):")
    cols_check = ['internacoes_mental', 'internacoes_agressao', 'internacoes_traumas_lesoes']
    for col in cols_check:
        if col in df_mestre.columns:
            print(f"   - {col}: {df_mestre[col].sum()}")
        else:
            print(f"   - {col}: NÃO EXISTE")

    return df_mestre

if __name__ == "__main__":
    df_final = gerar_painel_analitico_final()

🚀 Gerando Painel Analítico Final...
📊 Processando População...
💀 Processando SIM...
   Arquivo SIM utilizado: sim_mortalidade_externa_df.csv
🏥 Processando SIH...
   Arquivo SIH utilizado: sih_internacoes_externas_df.csv


/tmp/ipython-input-1532702421.py:95: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sih = pd.read_csv(caminho_sih)


🚓 Processando SINESP...
🤕 Processando SINAN...

🔗 Unificando tudo...
🧮 Calculando Indicadores por 100k habitantes...

🏆 SUCESSO! Painel Final salvo em: /content/drive/MyDrive/criminalidade_df/data/final/PAINEL_ANALITICO_FINAL.csv

🔎 Amostra das Colunas:
['ano', 'populacao', 'taxa_homicidio_saude', 'taxa_homicidio_policia', 'taxa_suicidio', 'taxa_internacao_mental', 'taxa_internacao_agressao', 'obitos_homicidio', 'obitos_outras_causas_externas', 'obitos_suicidio', 'obitos_transito', 'obitos_transtornos_mentais', 'internacoes_acidentes_outros', 'internacoes_agressao', 'internacoes_autoprovocada', 'internacoes_mental', 'internacoes_traumas_lesoes', 'custo_acidentes_outros', 'custo_agressao', 'custo_autoprovocada', 'custo_mental', 'custo_traumas_lesoes', 'ocorrencias_estupro', 'ocorrencias_furto_geral', 'ocorrencias_homicidio_doloso', 'ocorrencias_latrocinio', 'ocorrencias_outros_crimes', 'ocorrencias_roubo_geral', 'notificacoes_violencia']

🔎 Verificação de Conteúdo (Soma total dos anos):

In [3]:
import pandas as pd
import os

# --- CONFIGURAÇÕES ---
BASE_DIR = '/content/drive/MyDrive/criminalidade_df'
PROCESSED_DIR = os.path.join(BASE_DIR, 'data/processed')

def diagnostico_sih_profundo():
    print("🕵️‍♂️ Iniciando Diagnóstico Profundo do SIH...")

    # 1. Tenta carregar o arquivo "completo"
    f_candidates = [f for f in os.listdir(PROCESSED_DIR) if 'sih_' in f.lower() and 'externas' in f and 'csv' in f]

    if not f_candidates:
        print("❌ Arquivo 'sih_internacoes_externas_df.csv' não encontrado.")
        return

    arquivo_alvo = f_candidates[0]
    caminho = os.path.join(PROCESSED_DIR, arquivo_alvo)
    print(f"📂 Analisando arquivo: {arquivo_alvo}")

    df = pd.read_csv(caminho)

    # 2. Verifica se está vazio
    if df.empty:
        print("❌ O DataFrame está VAZIO! O problema foi na etapa de download/processamento anterior.")
        return

    print(f"✅ Total de Linhas no arquivo: {len(df)}")

    # 3. Analisa a coluna de Diagnóstico Principal
    col_diag = 'diag_principal_cid10'
    if col_diag not in df.columns:
        print(f"❌ Coluna '{col_diag}' não encontrada. Colunas: {df.columns.tolist()}")
        return

    print(f"\n🔬 Amostra de 10 Diagnósticos (CIDs) Brutos:")
    print(df[col_diag].head(10).tolist())

    # 4. Teste de Classificação (Simulando o Painel Final)
    def classificar_teste(cid):
        cid = str(cid).upper().strip()
        if cid.startswith('F'): return 'Mental'
        if cid.startswith(('X8', 'X9', 'Y0')): return 'Agressao'
        if cid.startswith(('X6', 'X7', 'X80', 'X81', 'X82', 'X83', 'X84')): return 'Autoprovocada'
        if cid.startswith(('S', 'T')): return 'Traumas_Lesoes'
        return 'Outros'

    df['grupo_teste'] = df[col_diag].apply(classificar_teste)

    print("\n📊 Contagem dos Grupos (Simulação):")
    contagem = df['grupo_teste'].value_counts()
    print(contagem)

    # 5. Caça aos CIDs perdidos (Verificar se existem X ou Y que não foram pegos)
    print("\n🔎 Verificando se existem CIDs de Agressão (X85-Y09) no arquivo...")
    # Filtra tudo que começa com X ou Y
    agressoes_reais = df[df[col_diag].astype(str).str.upper().str.startswith(('X', 'Y'))]

    if not agressoes_reais.empty:
        print(f"   ✅ Encontrados {len(agressoes_reais)} registros potenciais de agressão/suicídio.")
        print("   Exemplos:")
        print(agressoes_reais[col_diag].head(5).tolist())
    else:
        print("   ❌ ZERO registros começando com X ou Y encontrados. O filtro do download pode ter sido muito restrito.")

    # 6. Verificando Saúde Mental
    mentais = df[df[col_diag].astype(str).str.upper().str.startswith('F')]
    print(f"\n🔎 Verificando Saúde Mental (F)... Encontrados: {len(mentais)}")

diagnostico_sih_profundo()

🕵️‍♂️ Iniciando Diagnóstico Profundo do SIH...
📂 Analisando arquivo: sih_internacoes_externas_df.csv


/tmp/ipython-input-2504949490.py:22: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(caminho)


✅ Total de Linhas no arquivo: 178935

🔬 Amostra de 10 Diagnósticos (CIDs) Brutos:
['S061', 'S720', 'S920', 'S024', 'S272', 'S024', 'S122', 'S920', 'S525', 'S023']

📊 Contagem dos Grupos (Simulação):
grupo_teste
Traumas_Lesoes    140631
Mental             38276
Outros                25
Agressao               2
Autoprovocada          1
Name: count, dtype: int64

🔎 Verificando se existem CIDs de Agressão (X85-Y09) no arquivo...
   ✅ Encontrados 7 registros potenciais de agressão/suicídio.
   Exemplos:
['X939', 'X728', 'Y049', 'Y798', 'X040']

🔎 Verificando Saúde Mental (F)... Encontrados: 38276
